In [53]:
### imports

# external modules
import os
from os.path import exists
import os.path
import pandas as pd
import json
import sys
import itertools
import numpy as np
import matplotlib.pyplot as plt
import importlib
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.models import Model, Sequential, load_model
import importlib

# local modules
sys.path.append('../utils')
import csv_utils as csvu
import json_utils as jsonu
import dataframe_utils as dfu
import hist_utils as hu
import autoencoder_utils as aeu
import plot_utils as pu
import generate_data_utils as gdu
import refruns_utils as rru
importlib.reload(csvu)
importlib.reload(jsonu)
importlib.reload(dfu)
importlib.reload(hu)
importlib.reload(aeu)
importlib.reload(pu)
importlib.reload(gdu)
importlib.reload(rru)
sys.path.append('../src')
sys.path.append('../src/classifiers')
sys.path.append('../src/cloudfitters')
import HistStruct
importlib.reload(HistStruct)
import SubHistStruct
importlib.reload(SubHistStruct)
import DataLoader
importlib.reload(DataLoader)
import AutoEncoder
importlib.reload(AutoEncoder)
import SeminormalFitter
import GaussianKdeFitter
import HyperRectangleFitter
importlib.reload(SeminormalFitter)
importlib.reload(GaussianKdeFitter)
importlib.reload(HyperRectangleFitter)

<module 'HyperRectangleFitter' from '/eos/home-i01/k/khowey/SWAN_projects/ML4DQMDC-PixelAE/KH-AutoencoderTest/../src/cloudfitters/HyperRectangleFitter.py'>

In [43]:
year = '2017'
era = 'B'

datadir = '../data/' + year+era + '/'

In [44]:
blk1Vars = ['chargeInner', 'chargeOuter', 'adc', 'size']
blk2Vars = ['NormalizedHitResiduals', 'Summary_ClusterStoNCorr__OnTrack_',
            'Summary_TotalNumberOfDigis_']
blk3Vars = ['NumberOfTracks', 'NumberOfRecHitsPerTrack', 'Chi2oNDF',
            'goodvtxNbr'] 
miscVars = ['NumberOfClustersInPixel', 'num_clusters_ontrack_PXBarrel', 
            'num_clusters_ontrack_PXForward', 'NumberOfClustersInStrip']

In [45]:
### Get the different permutations for block 1
combosBlk1 = []
histcount = 0
modelcount = 0
for size in range(1, len(blk1Vars) + 1):
    # Get every combination of given size
    thisList = list(itertools.combinations(blk1Vars, size))
    
    
    ## Applying rules
    for item in thisList:
        
        if 'chargeInner' not in item: continue
        
        subList = []
        subListPX = []
        subListDSP = []
        subListDSN = []
        # Getting individual histograms to set appropriate names
        for element in item:
            
            # Need to treat chargeInner the same as 'charge' for disks
            if element == 'chargeInner':
                for i in range(1, 4):
                    subListDSP.append('charge_PXDisk_+' + str(i))
                    subListDSN.append('charge_PXDisk_-' + str(i))
                    
            elif element != 'chargeOuter':
                for i in range(1, 4):
                    subListDSP.append(element + '_PXDisk_+' + str(i))
                    subListDSN.append(element + '_PXDisk_-' + str(i))
            # PXlayers
            for i in range(1, 5):
                subListPX.append(element + '_PXLayer_' + str(i))
        
        subList.append(subListPX)
        subList.append(subListDSP)
        subList.append(subListDSN)
        
        combosBlk1.append(subList)

In [46]:
### Permutations for block 2
combosBlk2 = []
for size in range(1, len(blk2Vars) + 1):
    # Get every combination of given size
    thisList = list(itertools.combinations(blk2Vars, size))
    
    
    ## Applying rules
    for item in thisList:
        if 'Summary_ClusterStoNCorr__OnTrack_' not in item: continue
            
        subList = []
        subTIB = []
        subTOB = []
        subTIDP = []
        subTIDN = []
        subTECP = []
        subTECN = []
        
        # Getting individual histograms to set appropriate names
        for element in item:
            
            # Special case
            if element != 'NormalizedHitResiduals':
                for i in range(1, 10):
                    subTECN.append(element + '_TEC__MINUS__wheel__' + str(i))
                    subTECP.append(element + '_TEC__PLUS__wheel__' + str(i))
                    
                for i in range(1, 4):
                    subTIDN.append(element + '_TID__MINUS__wheel__' + str(i))
                    subTIDP.append(element + '_TID__PLUS__wheel__' + str(i))
                    
                for i in range(1, 5):
                    subTIB.append(element + '_TIB__layer__' + str(i))
            
                for i in range(1, 7):
                    subTOB.append(element + '_TOB__layer__' + str(i))
                    
            else: 
                for i  in range(1, 10):
                    subTECN.append(element + '_TEC__wheel__' + str(i))
                    
                for i in range(1, 4):
                    subTIDN.append(element + '_TID__wheel__' + str(i))
                    
                for i in range(1, 5):
                    subTIB.append(element + '_TIB__Layer__' + str(i))
            
                for i in range(1, 7):
                    subTOB.append(element + '_TOB__Layer__' + str(i))

        subList.append(subTIB)
        subList.append(subTOB)
        if len(subTIDP) > 0:
            subList.append(subTIDP)
        subList.append(subTIDN)
        if len(subTECP) > 0:
            subList.append(subTECP)
        subList.append(subTECN)
        combosBlk2.append(subList)

print(combosBlk2[0])

[['Summary_ClusterStoNCorr__OnTrack__TIB__layer__1', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__2', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__3', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__4'], ['Summary_ClusterStoNCorr__OnTrack__TOB__layer__1', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__2', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__3', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__4', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__5', 'Summary_ClusterStoNCorr__OnTrack__TOB__layer__6'], ['Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__1', 'Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__2', 'Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__3'], ['Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__1', 'Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__2', 'Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__3'], ['Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__1', 'Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__2', 'Summary_ClusterStoNC

In [47]:
### Permutations for block 3
combosBlk3 = []

for size in range(1, len(blk3Vars) + 1):
    # Get every combination of given size
    thisList = list(itertools.combinations(blk3Vars, size))
    
    ## Applying rules
    for item in thisList:
        if 'NumberOfRecHitsPerTrack' not in item: continue
         
        subList = []
        for element in item:
            if element !='goodvtxNbr':
                subList.append(element + '_lumiFlag_GenTk')
            else:
                subList.append(element)
        
        combosBlk3.append([subList])

print(combosBlk3[0])

[['NumberOfRecHitsPerTrack_lumiFlag_GenTk']]


In [48]:
### Permutations for block 4
combosBlk4 = []

for size in range(0, len(miscVars) + 1):
    # Get every combination of given size
    thisList = list(itertools.combinations(miscVars, size))
    
    ## Applying rules
    for item in thisList:
        subList = []
        subSubList = []
        for element in item:
            subSubList.append(element)
    
        subList.append(subSubList)
        combosBlk4.append(subList)

print(combosBlk4[0])

[[]]


In [51]:
### Parsing combinations to create histlists
histlists = []
conmodelcount = 0
combmodelcount = 0
for combo1 in combosBlk1:
    for combo2 in combosBlk2:
        for combo3 in combosBlk3:
            for combo4 in combosBlk4:
                curList = []
                for element in combo1:
                    curList.append(element)
                for element in combo2:
                    curList.append(element)
                for element in combo3:
                    curList.append(element)
                for element in combo4:
                    if len(element) > 0:
                        curList.append(element)
                
                # Sanity check that all files exist
                for histgroup in curList:
                    for hist in histgroup:
                        filename = 'DF' + year + era + '_' + hist +'.csv'
                        path = datadir + filename
                        if not os.path.exists(path):
                            raise Exception('Histogram {} does not exist!'.format(hist))
                histlists.append(curList)
                
for histlist in histlists:
    for histgroup in histlist:
        conmodelcount = conmodelcount + 1
        for hist in histgroup:
            combmodelcount = combmodelcount + 1
            
print('Models to Train:')
print(' - Concatamash: ' + str(conmodelcount))
print(' - Combined: ' + str(combmodelcount))

print('\nTraining Sets: ' + str(len(histlists)))

print('histnames = ' + str(histlists[4095]))

Models to Train:
 - Concatamash: 44800
 - Combined: 362496

Training Sets: 4096
histnames = [['chargeInner_PXLayer_1', 'chargeInner_PXLayer_2', 'chargeInner_PXLayer_3', 'chargeInner_PXLayer_4', 'chargeOuter_PXLayer_1', 'chargeOuter_PXLayer_2', 'chargeOuter_PXLayer_3', 'chargeOuter_PXLayer_4', 'adc_PXLayer_1', 'adc_PXLayer_2', 'adc_PXLayer_3', 'adc_PXLayer_4', 'size_PXLayer_1', 'size_PXLayer_2', 'size_PXLayer_3', 'size_PXLayer_4'], ['charge_PXDisk_+1', 'charge_PXDisk_+2', 'charge_PXDisk_+3', 'adc_PXDisk_+1', 'adc_PXDisk_+2', 'adc_PXDisk_+3', 'size_PXDisk_+1', 'size_PXDisk_+2', 'size_PXDisk_+3'], ['charge_PXDisk_-1', 'charge_PXDisk_-2', 'charge_PXDisk_-3', 'adc_PXDisk_-1', 'adc_PXDisk_-2', 'adc_PXDisk_-3', 'size_PXDisk_-1', 'size_PXDisk_-2', 'size_PXDisk_-3'], ['NormalizedHitResiduals_TIB__Layer__1', 'NormalizedHitResiduals_TIB__Layer__2', 'NormalizedHitResiduals_TIB__Layer__3', 'NormalizedHitResiduals_TIB__Layer__4', 'Summary_ClusterStoNCorr__OnTrack__TIB__layer__1', 'Summary_ClusterSto

In [57]:
### Data Controls and Selection - 1D Autoncoder

# The directory data is located in
datadir = '../data/'

# Select a list of good runs to train on in development training_mode
# Should be validated by eye
trainrunsls = {'2017B':{
#                    "297057":[[-1]], 
#                    "297099":[[-1]], 
#                    "297101":[[-1]],
#                    "297113":[[-1]], 
#                    "297114":[[-1]], 
                    "297175":[[-1]],   # A decently clean histogram
#                    "297177":[[-1]],
#                    "297179":[[-1]], 
#                    "297215":[[-1]],
#                    "297218":[[-1]],
#                    "297225":[[-1]],
#                    "297296":[[-1]], 
#                    "297411":[[-1]],
#                    "297426":[[-1]],  
#                    "297431":[[-1]],
#                    "297434":[[-1]], 
#                    "297468":[[-1]],
#                    "297483":[[-1]],
#                    "297486":[[-1]],
#                    "297503":[[-1]],
#                    "297557":[[-1]],
#                   "297598":[[-1]],
#                   "297604":[[-1]],   # A decently clean histogram
                   "297620":[[-1]],   # A decently clean histogram
                   "297659":[[-1]],   # An okay histogram
                   "297670":[[-1]],   # A decently clean histogram
#                    "297674":[[-1]],
#                    "297678":[[-1]],   # A particularly messy histogram
#                    "297722":[[-1]],   # A decently clean histogram
#                    "298997":[[-1]],
#                    "299061":[[-1]],
                   "299065":[[-1]],   # A decently clean histogram
                   "299067":[[-1]],   # A decently clean histogram
                   "299096":[[-1]],
                   "299149":[[-1]],
#                    "299178":[[-1]],   # A decently clean histogram
#                    "299184":[[-1]],   # A particularly messy histogram
#                    "299185":[[-1]],   # A decently clean histogram
#                    "299327":[[-1]],
#                    "299329":[[-1]], 
#                    "299480":[[-1]]    # A decently clean histogram
                    },
                   '2017C': {
#                      "299370":[[-1]],
#                      "299394":[[-1]],
#                      "299420":[[-1]],
#                      "299477":[[-1]],
#                      "299593":[[-1]],
#                      "299597":[[-1]],
#                      "299617":[[-1]],
#                      "300018":[[-1]],
#                      "300105":[[-1]],
#                      "300117":[[-1]],
#                      "300124":[[-1]],
#                      "300234":[[-1]],
#                      "300237":[[-1]],
#                      "300240":[[-1]],
#                      "300370":[[-1]],
#                      "300157":[[-1]],
#                      "300373":[[-1]],
#                      "300392":[[-1]],
#                      "300395":[[-1]],
#                      "300401":[[-1]],
#                      "300462":[[-1]],
#                      "300466":[[-1]],
                      "300514":[[-1]],
                      "300517":[[-1]],
                      "300538":[[-1]],
                      "300539":[[-1]],
                      "300364":[[-1]],
                 },'2017F':{
#                      "305310":[[-1]],
#                      "305040":[[-1]],
#                      "305043":[[-1]],
#                      "305185":[[-1]],
#                      "305204":[[-1]],
                      "305234":[[-1]],
                      "305247":[[-1]],
                      "305313":[[-1]],
                      "305338":[[-1]],
                      "305350":[[-1]],
                      "305364":[[-1]],
                      "305376":[[-1]],
                      "306042":[[-1]],
                      "306051":[[-1]],
                      "305406":[[-1]],
                      "306122":[[-1]],
                      "306134":[[-1]],
                      "306137":[[-1]],
                      "306154":[[-1]],
                      "306170":[[-1]],
                      "306417":[[-1]],
                      "306432":[[-1]],
                      "306456":[[-1]],
                      "305516":[[-1]],
                      "305586":[[-1]],
                      "305588":[[-1]],
                      "305590":[[-1]],
                      "305809":[[-1]],
                      "305832":[[-1]],
                      "305840":[[-1]],
                      "305898":[[-1]],
                      "306029":[[-1]],
                      "306037":[[-1]],
                      "306095":[[-1]],
                },
                '2018':{ # needs to be re-checked, not guaranteed to be fully correct or representative.   
                  "315267":[[-1]] 
              }
}

# Select a list of good runs to test on in development training_mode
# Should be validated by eye
goodrunsls = {'2017B':{
#                    "297057":[[-1]], 
#                    "297099":[[-1]], 
#                    "297101":[[-1]],
#                    "297113":[[-1]], 
#                    "297114":[[-1]], 
                    "297175":[[-1]],   # A decently clean histogram
#                    "297177":[[-1]],
#                    "297179":[[-1]], 
#                    "297215":[[-1]],
#                    "297218":[[-1]],
#                    "297225":[[-1]],
#                    "297296":[[-1]], 
#                    "297411":[[-1]],
#                    "297426":[[-1]],  
#                    "297431":[[-1]],
#                    "297434":[[-1]], 
#                    "297468":[[-1]],
#                    "297483":[[-1]],
#                    "297486":[[-1]],
#                    "297503":[[-1]],
#                    "297557":[[-1]],
#                    "297598":[[-1]],
#                    "297604":[[-1]],   # A decently clean histogram
#                    "297620":[[-1]],   # A decently clean histogram
                    "297659":[[-1]],   # An okay histogram
                    "297670":[[-1]],   # A decently clean histogram
                    "297674":[[-1]],
#                    "297678":[[-1]],   # A particularly messy histogram
                    "297722":[[-1]],   # A decently clean histogram
#                    "298997":[[-1]],
#                    "299061":[[-1]],
                    "299065":[[-1]],   # A decently clean histogram
                    "299067":[[-1]],   # A decently clean histogram
#                    "299096":[[-1]],
#                    "299149":[[-1]],
#                    "299178":[[-1]],   # A decently clean histogram
#                    "299184":[[-1]],   # A particularly messy histogram
                    "299185":[[-1]],   # A decently clean histogram
                    "299327":[[-1]],
#                    "299329":[[-1]], 
                    "299480":[[-1]]    # A decently clean histogram
                    },
                '2017C':{
#                      "299370":[[-1]],
#                      "299394":[[-1]],
#                      "299420":[[-1]],
#                      "299477":[[-1]],
#                      "299593":[[-1]],
#                      "299597":[[-1]],
#                      "299617":[[-1]],
#                      "300018":[[-1]],
#                      "300105":[[-1]],
#                      "300117":[[-1]],
#                      "300124":[[-1]],
#                      "300234":[[-1]],
#                      "300237":[[-1]],
#                      "300240":[[-1]],
#                      "300370":[[-1]],
#                      "300157":[[-1]],
#                      "300373":[[-1]],
#                      "300392":[[-1]],
#                      "300395":[[-1]],
#                      "300401":[[-1]],
#                      "300462":[[-1]],
#                      "300466":[[-1]],
                      "300514":[[-1]],
                      "300517":[[-1]],
                      "300538":[[-1]],
                      "300539":[[-1]],
                      "300364":[[-1]],
                },'2017F':{
                      "305310":[[-1]],
                      "305040":[[-1]],
                      "305043":[[-1]],
                      "305185":[[-1]],
                      "305204":[[-1]],
#                      "305234":[[-1]],
#                      "305247":[[-1]],
#                      "305313":[[-1]],
#                      "305338":[[-1]],
#                      "305350":[[-1]],
#                      "305364":[[-1]],
#                      "305376":[[-1]],
#                      "306042":[[-1]],
#                      "306051":[[-1]],
#                      "305406":[[-1]],
#                      "306122":[[-1]],
#                      "306134":[[-1]],
#                      "306137":[[-1]],
#                      "306154":[[-1]],
#                      "306170":[[-1]],
#                      "306417":[[-1]],
#                      "306432":[[-1]],
#                      "306456":[[-1]],
#                      "305516":[[-1]],
#                      "305586":[[-1]],
#                      "305588":[[-1]],
#                      "305590":[[-1]],
#                      "305809":[[-1]],
#                      "305832":[[-1]],
#                      "305840":[[-1]],
#                      "305898":[[-1]],
#                      "306029":[[-1]],
#                      "306037":[[-1]],
#                      "306095":[[-1]],
                },
                '2018':{ # needs to be re-checked, not guaranteed to be fully correct or representative.   
                  "315267":[[-1]] 
              }
}


badrunsls = {'2017B':
                {
                    #"297048":[[-1]],
                    #"297282":[[-1]],
                    #"297283":[[-1]],
                    #"297284":[[-1]],
                    #"297287":[[-1]],
                    #"297288":[[-1]],
                    #"297289":[[-1]],
                    "299316":[[-1]],
                    "299317":[[-1]],
                    "299318":[[-1]],
                    "299324":[[-1]],
                    "299326":[[-1]],
                    #"301086":[[88,126]],
                    #"301086":[[89,89]],
                    #"303948":[[1710,1710]],
                    "297047":[[-1]], #close but, true bad for all 8
                    "297169":[[-1]], #true bad for all 8
#                   "297211":[[-1]], #Reconstructs well
#                   "299325":[[-1]], #Reconstructs well
                    "297664":[[-1]], #true bad for all 8
                    "299317":[[-1]], #true bad for all 8
                    "297169":[[-1]], #true bad for all 8
#                   "297502":[[-1]]
                },
             '2017C':{
                "300079":[[-1]],
                "300282":[[-1]],
                "300389":[[-1]],
                "300398":[[-1]],
                 
                 
#                 "300781":[[-1]], # bad for tracking (pixels were excluded.
#                 "300079":[[-1]], # is bad for strips and then also for tracking
#                 "302029":[[-1]], # Poor detector elements for strips - Should be mildly anomalous, but technically good 
#                 "300576":[[-1]], # Poor detector elements for strips - Should be mildly anomalous, but technically good
#                 "300574":[[-1]], # Poor detector elements for strips - Should be mildly anomalous, but technically good
#                 "300282":[[-1]], # Poor detector elements for strips - Should be mildly anomalous, but technically good
#                 "301912":[[-1]], # Half bad for pixels (lost HV or readout card)  
#                 "301086":[[-1]], # Half bad for pixels (lost HV or readout card)  
#                 "300283":[[-1]], # Half bad for pixels (lost HV or readout card) 
#                 "300282":[[-1]], # Half bad for pixels (lost HV or readout card) 
#                 "300281":[[-1]], # Half bad for pixels (lost HV or readout card) 
#                 "300239":[[-1]], # Half bad for pixels (lost HV or readout card)
#                 "301394":[[-1]], # Marginal for pixels
#                 "301183":[[-1]], # Marginal for pixels
#                 "300398":[[-1]], # Marginal for pixels
#                 "300389":[[-1]], # Marginal for pixels
#                 "300365":[[-1]]  # Marginal for pixels
              },
             '2017E':{
                 "304740":[[-1]], # Bad for pixels and tracking - holes in PXLayer 1
                 "304776":[[-1]], # Bad for pixels and tracking - holes in PXLayer 1
                 "304506":[[-1]], # Portcard problem for pixels
                 "304507":[[-1]], # Portcard problem for pixels 
                 "303989":[[-1]], # Bad for pixels, power supply died
                 "303824":[[-1]]  # Partly bad for strips due to a test
             },
             '2017F':{
                 "306422":[[-1]], # Partly bad for strips - 2 data readouts failed 
#                 "306423":[[-1]], # Partly bad for strips - 2 data readouts failed
#                 "306425":[[-1]], # Partly bad for strips - 2 data readouts failed
#                 "305440":[[-1]], # Partly bad for strips - 1 data readout failed
#                 "305441":[[-1]], # Partly bad for strips - 1 data readout failed
                 "305249":[[-1]], # Bad for pixels - half of disk failed 
                 "305250":[[-1]], # Bad for pixels - half of disk failed
#                 "305064":[[-1]], # Marginal for pixels - some readout failed
             },
            '2018': # needs to be re-checked, not guaranteed to be fully correct or representative.
                {
                #"317479":[[-1]],
                "317480":[[-1]],
                "317481":[[-1]],
                "317482":[[-1]],
                #"319847":[[1,35]]
            }}

# Create a list of histograms to include
# Pair histograms to be combined on the same line
histnames = histlists[len(histlists) - 1]

In [58]:
### Define Run Properties - Combined Autoencoder
# in this cell all major run properties are going to be set,

# Select whether to save a new histstruct
save = False

In [59]:
### Model Controls and Selection - 1D Autoencoder

plotNames = 'Test'
name = plotNames+'plots'

# Choose whether to train a new model or load one
trainnew = True
savemodel = True
modelname = plotNames

In [60]:
# train on a user-defined subset of runs
    
# Select runs to be used in training from the user-defined list
runsls_training = trainrunsls[year + era]
# Select bad runs to test on in the user-defined list
runsls_bad = badrunsls[year + era]
# Select good runs to test on in the user-defined list
runsls_good = goodrunsls[year + era]

print('selected runs/lumisections for training: ')
print(runsls_training)
print('selected runs/lumisections as good test set:')
print(runsls_good)
print('selected runs/lumisections as bad test set:')
print(runsls_bad)

selected runs/lumisections for training: 
{'297175': [[-1]], '297620': [[-1]], '297659': [[-1]], '297670': [[-1]], '299065': [[-1]], '299067': [[-1]], '299096': [[-1]], '299149': [[-1]]}
selected runs/lumisections as good test set:
{'297175': [[-1]], '297659': [[-1]], '297670': [[-1]], '297674': [[-1]], '297722': [[-1]], '299065': [[-1]], '299067': [[-1]], '299185': [[-1]], '299327': [[-1]], '299480': [[-1]]}
selected runs/lumisections as bad test set:
{'299316': [[-1]], '299317': [[-1]], '299318': [[-1]], '299324': [[-1]], '299326': [[-1]], '297047': [[-1]], '297169': [[-1]], '297664': [[-1]]}


In [62]:
# Initializations
dloader = DataLoader.DataLoader()
histstruct = SubHistStruct.SubHistStruct()
histstruct.reset_histlist(histnames)

# Unpack histnames and add every histogram individually
for histnamegroup in histnames:
    for histname in histnamegroup:
        print('Adding {}...'.format(histname))
        
        # Bring the histograms into memory from storage for later use
        filename = datadir + year + era + '/DF' + year + era + '_' + histname + '.csv'
        df = dloader.get_dataframe_from_file( filename )
        
        # In case of local training, we can remove most of the histograms
        if( runsls_training is not None and runsls_good is not None and runsls_bad is not None ):
            runsls_total = {k: v for d in (runsls_training, runsls_good, runsls_bad) for k, v in d.items()}
            df = dfu.select_runsls( df, runsls_total )
        
        df = dfu.rm_duplicates(df)
        
        try:
            # Store the data in the histstruct object managing this whole thing
            histstruct.add_dataframe( df, rebinningfactor = 1, standardbincount = 102 )
        except:
            print("WARNING: Could not add " + histname, file=sys.stderr)
    
print('Found {} histograms'.format(len(histstruct.runnbs)))
print('Created a histstruct with the following properties:')
print('- number of histogram types: {}'.format(len(histstruct.histnames)))
print('- number of lumisections: {}'.format(len(histstruct.lsnbs)))

Adding chargeInner_PXLayer_1...
Adding chargeInner_PXLayer_2...
Adding chargeInner_PXLayer_3...
Adding chargeInner_PXLayer_4...
Adding chargeOuter_PXLayer_1...
Adding chargeOuter_PXLayer_2...
Adding chargeOuter_PXLayer_3...
Adding chargeOuter_PXLayer_4...
Adding adc_PXLayer_1...
Adding adc_PXLayer_2...
Adding adc_PXLayer_3...
Adding adc_PXLayer_4...
Adding size_PXLayer_1...
Adding size_PXLayer_2...
Adding size_PXLayer_3...
Adding size_PXLayer_4...
Adding charge_PXDisk_+1...
Adding charge_PXDisk_+2...
Adding charge_PXDisk_+3...
Adding adc_PXDisk_+1...
Adding adc_PXDisk_+2...
Adding adc_PXDisk_+3...
Adding size_PXDisk_+1...
Adding size_PXDisk_+2...
Adding size_PXDisk_+3...
Adding charge_PXDisk_-1...
Adding charge_PXDisk_-2...
Adding charge_PXDisk_-3...
Adding adc_PXDisk_-1...
Adding adc_PXDisk_-2...
Adding adc_PXDisk_-3...
Adding size_PXDisk_-1...
Adding size_PXDisk_-2...
Adding size_PXDisk_-3...
Adding NormalizedHitResiduals_TIB__Layer__1...
Adding NormalizedHitResiduals_TIB__Layer__2..

Adding NormalizedHitResiduals_TOB__Layer__2...
Adding NormalizedHitResiduals_TOB__Layer__3...
Adding NormalizedHitResiduals_TOB__Layer__4...
Adding NormalizedHitResiduals_TOB__Layer__5...
Adding NormalizedHitResiduals_TOB__Layer__6...
Adding Summary_ClusterStoNCorr__OnTrack__TOB__layer__1...
Adding Summary_ClusterStoNCorr__OnTrack__TOB__layer__2...
Adding Summary_ClusterStoNCorr__OnTrack__TOB__layer__3...
Adding Summary_ClusterStoNCorr__OnTrack__TOB__layer__4...
Adding Summary_ClusterStoNCorr__OnTrack__TOB__layer__5...
Adding Summary_ClusterStoNCorr__OnTrack__TOB__layer__6...
Adding Summary_TotalNumberOfDigis__TOB__layer__1...
Adding Summary_TotalNumberOfDigis__TOB__layer__2...
Adding Summary_TotalNumberOfDigis__TOB__layer__3...
Adding Summary_TotalNumberOfDigis__TOB__layer__4...
Adding Summary_TotalNumberOfDigis__TOB__layer__5...


Adding Summary_TotalNumberOfDigis__TOB__layer__6...
Adding Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__1...


Adding Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__2...
Adding Summary_ClusterStoNCorr__OnTrack__TID__PLUS__wheel__3...
Adding Summary_TotalNumberOfDigis__TID__PLUS__wheel__1...
Adding Summary_TotalNumberOfDigis__TID__PLUS__wheel__2...
Adding Summary_TotalNumberOfDigis__TID__PLUS__wheel__3...
Adding NormalizedHitResiduals_TID__wheel__1...


Adding NormalizedHitResiduals_TID__wheel__2...
Adding NormalizedHitResiduals_TID__wheel__3...
Adding Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__1...
Adding Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__2...
Adding Summary_ClusterStoNCorr__OnTrack__TID__MINUS__wheel__3...
Adding Summary_TotalNumberOfDigis__TID__MINUS__wheel__1...
Adding Summary_TotalNumberOfDigis__TID__MINUS__wheel__2...
Adding Summary_TotalNumberOfDigis__TID__MINUS__wheel__3...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__PLUS__wheel__1...


...

Adding Summary_TotalNumberOfDigis__TEC__PLUS__wheel__6...
Adding Summary_TotalNumberOfDigis__TEC__PLUS__wheel__7...
Adding Summary_TotalNumberOfDigis__TEC__PLUS__wheel__8...
Adding Summary_TotalNumberOfDigis__TEC__PLUS__wheel__9...
Adding NormalizedHitResiduals_TEC__wheel__1...


Adding NormalizedHitResiduals_TEC__wheel__2...
Adding NormalizedHitResiduals_TEC__wheel__3...
Adding NormalizedHitResiduals_TEC__wheel__4...
Adding NormalizedHitResiduals_TEC__wheel__5...
Adding NormalizedHitResiduals_TEC__wheel__6...
Adding NormalizedHitResiduals_TEC__wheel__7...
Adding NormalizedHitResiduals_TEC__wheel__8...
Adding NormalizedHitResiduals_TEC__wheel__9...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__1...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__2...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__3...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__4...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__5...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__6...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__7...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__8...
Adding Summary_ClusterStoNCorr__OnTrack__TEC__MINUS__wheel__9...
Adding Summary_TotalNumberOfDigis__TEC_

Adding Summary_TotalNumberOfDigis__TEC__MINUS__wheel__6...
Adding Summary_TotalNumberOfDigis__TEC__MINUS__wheel__7...
Adding Summary_TotalNumberOfDigis__TEC__MINUS__wheel__8...
Adding Summary_TotalNumberOfDigis__TEC__MINUS__wheel__9...
Adding NumberOfTracks_lumiFlag_GenTk...


Adding NumberOfRecHitsPerTrack_lumiFlag_GenTk...
Adding Chi2oNDF_lumiFlag_GenTk...
Adding goodvtxNbr...
Adding NumberOfClustersInPixel...


Adding num_clusters_ontrack_PXBarrel...
Adding num_clusters_ontrack_PXForward...
Adding NumberOfClustersInStrip...
Found 2831 histograms
Created a histstruct with the following properties:
- number of histogram types: 95
- number of lumisections: 2831


In [ ]:
### Add Masks to Data

histstruct.add_dcsonjson_mask( 'dcson' )
histstruct.add_goldenjson_mask('golden' )
histstruct.add_highstat_mask( 'highstat' )
histstruct.add_stat_mask( 'lowstat', max_entries_to_bins_ratio=100 )
if runsls_training is not None: histstruct.add_json_mask( 'training', runsls_training )
if runsls_good is not None: histstruct.add_json_mask( 'good', runsls_good )
    
# Distinguishing bad runs
nbadruns = 0
if runsls_bad is not None:
    print(runsls_bad)
    histstruct.add_json_mask( 'bad', runsls_bad )
    
    # Special case for bad runs: add a mask per run (different bad runs have different characteristics)
    nbadruns = len(runsls_bad.keys())
    for i,badrun in enumerate(runsls_bad.keys()):
        histstruct.add_json_mask( 'bad{}'.format(i), {badrun:runsls_bad[badrun]} )
        
if save:
    histstruct.save('test.pk1')
if userfriendly: print('Assigned masks: {}'.format(list(histstruct.masks.keys())))

In [ ]:
def define_concatamash_autoencoder(histstruct):
    
    histslist = []
    vallist = []
    autoencoders = []
    if trainnew:
        for i,histnamegroup in enumerate(histnames):
            
            train_normhist = np.array([hu.normalize(histstruct.get_histograms(
                histname = hname, masknames = ['dcson','highstat', 'training']), 
                                                 norm="l1", axis=1) 
                                       for hname in histnamegroup]).transpose((1,0,2))
            X_train, X_val = train_test_split(train_normhist, test_size=0.4, random_state=42)
            
            print('\nNow Defining model {}/'.format(i + 1) 
                  + str(len(histnames)))
            print(' - Size of training set: {}'.format(X_train.shape))
            
            # Half the total bin count
            arch = 51 * len(histnamegroup)
            
            ## Model parameters
            print(X_train.shape)
            
            input_dim = X_train.shape[2] #num of predictor variables
            Input_layers=[Input(shape=input_dim) for i in range((X_train.shape[1]))]
            
            # Defining layers
            conc_layer = Concatenate()(Input_layers)
            encoder = Dense(arch * 2, activation="tanh")(conc_layer)
            #encoder = Dense(128, activation='relu')(encoder)
            #
            #encoder = Dense(32, activation='relu')(encoder)
            
            decoder = Dense(arch, activation="relu")(encoder)
            #decoder = Dense(256, activation="tanh")(decoder)
            
            Output_layers=[Dense(input_dim, activation="tanh")(decoder) for i in range(X_train.shape[1])]

            autoencoder = Model(inputs=Input_layers, outputs=Output_layers)
            autoencoder.summary()
            autoencoders.append(autoencoder)
            
            histslist.append(X_train)
            vallist.append(X_val)
     
    # Return the histograms stored 2-Dimensionally and the autoencoders corresponding
    return(histslist, vallist, autoencoders, train_normhist)

In [ ]:
### Main loop to iterate through possible histlists
for histnames in histlists:
    histstruct.reset_histlist(histnames)
    print(str(histstruct))